In [ ]:
%matplotlib widget
import numpy as np
from matplotlib import pyplot as plt
from diffraction_simulation import diffraction_simulation as dsim
from diffraction_simulation.misc import fig2clipboard

dsim.StructureHandler.LENGTH_UNIT = 'angstrom'

ewald = dsim.EwaldSphere(300e3)
p21c   = dsim.StructureHandler.from_matproj('HfO2_P21c')
pca21  = dsim.StructureHandler.from_matproj('HfO2_Pca21')
p42nmc = dsim.StructureHandler.from_matproj('HfO2_P42nmc').BCT2FCC_transform()
TiN    = dsim.StructureHandler.from_matproj('TiN_Fm3m')
p21c.align_structure_uvw_to_z([0,0,-1])

fig = plt.figure(figsize=(12,4))
ax1 = fig.add_subplot(131, projection='3d', proj_type='ortho')
p21c.plot_unit_cell_3d(ax1, low_frac=(-0.1,-0.1,-0.01))
ax1.legend()

ax2 = fig.add_subplot(132, projection='3d', proj_type='ortho')
pca21.plot_unit_cell_3d(ax2, high_frac=(1.1, 1.1, 1.05))
ax2.legend()

ax3 = fig.add_subplot(133, projection='3d', proj_type='ortho')
p42nmc.plot_unit_cell_3d(ax3)
ax3.legend()

In [ ]:
from matplotlib.animation import FuncAnimation
fig = plt.figure(figsize=(11,4))
ax1 = fig.add_subplot(131, projection='3d', proj_type='ortho')
p21c.plot_unit_cell_3d(ax1, low_frac=(-0.1,-0.1,-0.01))
ax1.set_xlim(-1,6)
ax1.set_ylim(-1,6)
ax1.set_zlim(-1.5,6)

ax2 = fig.add_subplot(132, projection='3d', proj_type='ortho')
pca21.plot_unit_cell_3d(ax2, high_frac=(1.1, 1.1, 1.05))
ax2.set_xlim(-1,6)
ax2.set_ylim(-1,6)
ax2.set_zlim(-1.5,6)

ax3 = fig.add_subplot(133, projection='3d', proj_type='ortho')
p42nmc.plot_unit_cell_3d(ax3)
ax3.set_xlim(-1,6)
ax3.set_ylim(-1,6)
ax3.set_zlim(-1.5,6)

fig.tight_layout()
motion_time = 2
hold_time = 4
frame_time = 0.1
def anim(t):
    if t<motion_time:
        azim = 90*t/motion_time
        elev = 0
    elif t<motion_time+hold_time:
        azim = 90
        elev = 0
    elif t<2*motion_time+hold_time:
        azim = 90
        elev = 90*(t-motion_time-hold_time)/motion_time
    elif t<2*motion_time+2*hold_time:
        azim = 90
        elev = 90
    elif t<=3*motion_time+2*hold_time:
        elev = 90 - 90*(t-2*motion_time-2*hold_time)/motion_time
        azim = 90 - 90*(t-2*motion_time-2*hold_time)/motion_time
    else:
        elev = azim = 0
    for ax in (ax1,ax2,ax3):
        ax.view_init(elev=elev,azim=azim,roll=0)

T = 3*(motion_time+hold_time)
ani = FuncAnimation(fig, anim, 
                    frames=np.arange(0,T,frame_time),
                    interval=int(1000*frame_time), 
                    blit=False) # blit=True for faster rendering
ani.save('./HfO2-similarities.gif', dpi=50)
plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(12,4))
ax1 = fig.add_subplot(131)
pca21.plot_unit_cell_2d(ax1, high_frac=(1.1, 1.1, 1.05), proj_ax_uvw=(0,0,1), x_ax_uvw=(1,0,0))
ax1.set_xlabel('x')
ax1.set_ylabel('y')
# ax1.legend()

ax2 = fig.add_subplot(132)
pca21.plot_unit_cell_2d(ax2, high_frac=(1.1, 1.1, 1.05), proj_ax_uvw=(0,-1,0), x_ax_uvw=(1,0,0))
ax2.set_xlabel('x')
ax2.set_ylabel('z')
# ax2.legend()

ax3 = fig.add_subplot(133)
pca21.plot_unit_cell_2d(ax3, high_frac=(1.1, 1.1, 1.05), proj_ax_uvw=(1,0,0), x_ax_uvw=(0,1,0))
ax3.set_xlabel('y')
ax3.set_ylabel('z')
# ax3.legend()

# Diffraction

In [ ]:
dsim.StructureHandler.LENGTH_UNIT = 'nm'
dps = {}
num_orientations = 10000
g_max = 5*np.sqrt(2)
sg_max = 0.1

dps['p21c'] = p21c.powder_hkl(ewald, sg_max, g_max, num_orientations=num_orientations)
dps['pca21'] = pca21.powder_hkl(ewald, sg_max, g_max, num_orientations=num_orientations)
dps['p42nmc'] = p42nmc.powder_hkl(ewald, sg_max, g_max, num_orientations=num_orientations)
dps['TiN'] = TiN.powder_hkl(ewald, sg_max, g_max, num_orientations=num_orientations)

In [ ]:
dq = 0.05

fig,ax = plt.subplots(1,3,figsize=(12,4))
for ax, (label, dp) in zip(ax, dps.items()):
    dp.hist2d(ax, bins=200, sample_thickness=10, norm='log', cmap='gray')
    ax.set_aspect(1)

fig.tight_layout()
# fig2clipboard(fig, 400)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,2))

bins = np.linspace(1.9, g_max, 100)
for label, dp in dps.items():
    dp.hist(ax, sample_thickness=10, bins=bins, alpha=0.5, label=label)

ax.set_ylabel('Intensity (a.u.)')
ax.legend()
ax.minorticks_on()
# fig2clipboard(fig, 400)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(9,2))
bins = np.linspace(1.95, g_max, 50)
dsim.ElectronDiffractionSpots.stacked_hist(ax, *dps.values(), sample_thickness=10, bins=bins,
                                           color=['r','lime','b','orange'], 
                                           label=['mono','ortho','tetra', 'TiN'])

ax.set_ylabel('Stacked intensity (a.u.)')
ax.grid()
ax.legend(framealpha=1)
ax.minorticks_on()
fig2clipboard(fig, 400)

In [ ]:
theta,phi = dsim.StructureHandler._uniformly_sampled_points_on_sphere(500)
x,y,z = np.sin(theta)*np.sin(phi), np.sin(theta)*np.cos(phi), np.cos(theta)

fig = plt.figure()
ax = fig.add_subplot(projection='3d', proj_type='ortho')
ax.quiver(0, 0, 0, 1, 0, 0, color='r', length=0.5, normalize=True)
ax.quiver(0, 0, 0, 0, 1, 0, color='g', length=0.5, normalize=True)
ax.quiver(0, 0, 0, 0, 0, 1, color='b', length=0.5, normalize=True)
ax.grid(False)
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = False
ax.xaxis.pane.set_edgecolor('w')
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor('w')
ax.scatter(x,y,z)
ax.set_aspect('equal')


from matplotlib.animation import FuncAnimation

N = 100
fps = 120
def anim(t):
    azim = 360/N*t
    elev = 720/N*t
    ax.view_init(elev=elev,azim=azim,roll=0)
    fig.tight_layout()

ani = FuncAnimation(fig, anim, 
                    frames=N,
                    interval=int(1000/fps), 
                    blit=True) # blit=True for faster rendering
ani.save('./Uniform-Sphere.gif', dpi=100)
plt.close(fig)